In [143]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import matplotlib.pyplot as plt

import folium
import rasterio
import rasterio.mask

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from geopy.distance import geodesic
import folium
from sklearn.tree import DecisionTreeClassifier

In [144]:
incoming_weather_size_draught = pd.read_csv('Data/Input_ML/incoming_weather_size_draught.csv')
incoming_weather_size = pd.read_csv('Data/Input_ML/incoming_weather_size.csv')
incoming_weather = pd.read_csv('Data/Input_ML/incoming_weather.csv')

In [145]:
# Assuming 'column_name' is the name of the column containing strings in your DataFrame
label_encoder = LabelEncoder()
incoming_weather_size_draught['Direction'] = label_encoder.fit_transform(incoming_weather_size_draught['Direction'])
incoming_weather_size_draught['Wind_speed'] = label_encoder.fit_transform(incoming_weather_size_draught['Wind_speed'])
incoming_weather_size_draught['Type'] = label_encoder.fit_transform(incoming_weather_size_draught['Type'])
#incoming_weather_size_draught['to_haven'] = label_encoder.fit_transform(incoming_weather_size_draught['to_haven'])
incoming_weather_size_draught

,Unnamed: 0,area_ID,geometry,center,index_right,from,from_rounded,from_location,to_haven,trip_ID,...,Datetime,Direction,Wind_speed,device_mmsi_x,Lenght,Width,Type,device_mmsi_y,Navigation_draught,DateTime
0,0,17241,"POLYGON ((4.141337398344932 51.9668754358794, ...",POINT (4.142291272953951 51.965921561270385),8,2022-05-31 23:34:58+00:00,2022-06-01 00:00:00+00:00,"[4.14163, 51.96557]",3PET,4,...,2022-06-01 00:00:00+00:00,6,2,538005865.0,159.0,27.0,2,538005865,8.3,2022-06-01 00:00:00+00:00
1,1,28292,"POLYGON ((4.301588332660119 51.89628871481187,...",POINT (4.302542207269138 51.89533484020285),10632,2022-09-08 07:01:29+00:00,2022-09-08 07:00:00+00:00,"[4.30278, 51.89511]",1PET,6054,...,2022-09-08 07:00:00+00:00,5,3,538005865.0,159.0,27.0,2,538005865,9.6,2022-09-08 07:00:00+00:00
2,2,19867,"POLYGON ((4.179492382705691 51.94016694682682,...",POINT (4.18044625731471 51.9392130722178),10997,2022-09-11 04:34:51+00:00,2022-09-11 05:00:00+00:00,"[4.17956, 51.93872]",CKBRUG,6258,...,2022-09-11 05:00:00+00:00,0,2,538005865.0,159.0,27.0,2,538005865,9.0,2022-09-11 05:00:00+00:00
3,3,25526,POLYGON ((4.261525599081322 51.907735210120116...,POINT (4.262479473690341 51.9067813355111),20123,2022-11-24 02:30:45+00:00,2022-11-24 03:00:00+00:00,"[4.26256, 51.90621]",3PET,11529,...,2022-11-24 03:00:00+00:00,7,1,538005865.0,159.0,27.0,2,538005865,7.6,2022-11-24 03:00:00+00:00
4,4,26974,"POLYGON ((4.282510840479739 51.90010421324795,...",POINT (4.283464715088758 51.899150338638925),14458,2022-10-10 08:13:47+00:00,2022-10-10 08:00:00+00:00,"[4.28256, 51.89866]",TORONT,8267,...,2022-10-10 08:00:00+00:00,5,3,538005077.0,158.0,27.0,2,538005077,10.3,2022-10-10 08:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4042,4042,18289,"POLYGON ((4.156599392089236 51.95161344213507,...",POINT (4.157553266698255 51.95065956752606),23823,2022-12-25 00:56:07+00:00,2022-12-25 01:00:00+00:00,"[4.15802, 51.95009]",7PET,13637,...,2022-12-25 01:00:00+00:00,5,3,538003962.0,183.0,32.0,2,538003962,11.3,2022-12-25 01:00:00+00:00
4043,4043,11443,"POLYGON ((4.057396432751263 51.98595292805982,...",POINT (4.058350307360282 51.9849990534508),23850,2022-12-25 08:17:19+00:00,2022-12-25 08:00:00+00:00,"[4.05757, 51.98543]",SCHEUR,13651,...,2022-12-25 08:00:00+00:00,5,3,477345300.0,225.0,32.0,0,477345300,14.0,2022-12-25 08:00:00+00:00
4044,4044,13814,"POLYGON ((4.091735918675946 51.97641418196961,...",POINT (4.092689793284965 51.97546030736059),23857,2022-12-25 10:19:15+00:00,2022-12-25 10:00:00+00:00,"[4.09324, 51.97466]",5PET,13655,...,2022-12-25 10:00:00+00:00,5,3,538008798.0,245.0,42.0,2,538008798,15.3,2022-12-25 10:00:00+00:00
4045,4045,28160,"POLYGON ((4.299680583442081 51.89628871481187,...",POINT (4.3006344580511 51.89533484020285),23867,2022-12-25 12:25:08+00:00,2022-12-25 12:00:00+00:00,"[4.30134, 51.89561]",2PET,13660,...,2022-12-25 12:00:00+00:00,5,3,403579000.0,183.0,32.0,2,403579000,11.4,2022-12-25 12:00:00+00:00


In [146]:
leaving_weather_size_draught = pd.read_csv('Data/Input_ML/leaving_weather_size_draught.csv')
leaving_weather_size = pd.read_csv('Data/Input_ML/leaving_weather_size.csv')
leaving_weather = pd.read_csv('Data/Input_ML/leaving_weather.csv')

In [147]:
# Assuming 'column_name' is the name of the column containing strings in your DataFrame
label_encoder = LabelEncoder()
leaving_weather_size_draught['Direction'] = label_encoder.fit_transform(leaving_weather_size_draught['Direction'])
leaving_weather_size_draught['Wind_speed'] = label_encoder.fit_transform(leaving_weather_size_draught['Wind_speed'])
leaving_weather_size_draught['Type'] = label_encoder.fit_transform(leaving_weather_size_draught['Type'])
#leaving_weather_size_draught['from_haven'] = label_encoder.fit_transform(leaving_weather_size_draught['from_haven'])

In [148]:
leaving_top = leaving_weather_size_draught['from_haven'].value_counts()
incoming_top = incoming_weather_size_draught['to_haven'].value_counts()
print(leaving_top[0:2])
print(incoming_top[0:2])

from_haven
EURO      451
AMALIA    309
Name: count, dtype: int64
to_haven
EURO    480
7PET    377
Name: count, dtype: int64


In [149]:
top_leaving = 'EURO'
top_incoming = 'EURO'

harbor_leaving = leaving_weather_size_draught[leaving_weather_size_draught['from_haven']== top_leaving]
harbor_incoming = incoming_weather_size_draught[incoming_weather_size_draught['to_haven']== top_incoming]


In [150]:
columns_leaving = [
    #'from_haven',
    'Direction',
    'Wind_speed',
    'Lenght'	,
    'Width'	,
    'Type'
]

columns_incoming = [
    #'to_haven',
    'Direction',
    'Wind_speed',
    'Lenght'	,
    'Width'	,
    'Type'
]


In [151]:
X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(harbor_leaving[columns_leaving], harbor_leaving['area_ID'], test_size=0.3, random_state=42)
#X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(leaving_weather_size_draught[columns_leaving], leaving_weather_size_draught['area_ID'], test_size=0.3, random_state=42)

In [152]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Initialize the Decision Tree Classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Train the classifier using the training data
decision_tree_classifier.fit(X_from_lat_train, y_from_lat_train)

# Make predictions on the test data
predictions = decision_tree_classifier.predict(X_from_lat_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_from_lat_test, predictions)

print("Accuracy of the Decision Tree Classifier: {:.2f}%".format(accuracy * 100))


Accuracy of the Decision Tree Classifier: 2.94%


In [153]:
from math import radians, sin, cos, sqrt, atan2

def calculate_haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    radius = 6371.0
    distance = radius * c

    return distance

In [154]:
def get_polygon_center_from_id(polygon_id, polygon_gdf):
   
    polygon = polygon_gdf[polygon_gdf['ID'] == polygon_id].geometry.iloc[0]
    center = polygon.centroid

    return center.y, center.x  

In [156]:
predicted_distances = []

for predicted_polygon_id in predictions:
    # Calculate the distance using the Haversine formula
    distance = calculate_haversine_distance(incoming_weather_size_draught['center'][i][0], incoming_weather_size_draught['center'][i][1], predicted_polygon_center[0], predicted_polygon_center[1])
    
    # Append the distance to the list of predicted_distances
    predicted_distances.append(distance)

NameError: name 'i' is not defined